In [1]:
import torchvision.transforms as transforms
from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
from device import fetchDevice
import torch
import os
import fnmatch
import re
from PIL import Image

def list_files_by_type(folder_path, file_type):
        filtered_files = []
        for file in os.listdir(folder_path):
            if fnmatch.fnmatch(file, f"*.{file_type}"):
                filtered_files.append(os.path.join(folder_path, file))
        return filtered_files

def process_image(args):
    file_path, label, transform = args  # Unpack the tuple
    image = Image.open(file_path)
    default_transform = transforms.ToTensor()
    tensor = default_transform(image)
    if transform:
        tensor = transform(tensor)
    return tensor, torch.tensor(label, dtype=torch.long)

def extract_index_and_category(file_path):
    filename = os.path.basename(file_path)
    pattern = r"(\d+)(?: \((\d+)\))?\..+"
    match = re.match(pattern, filename)
    if match:
        index = int(match.group(1))
        category = int(match.group(2)) if match.group(2) else 0
        return index, category
    else:
        return None

def load_folder(folder_path, label, category=None, transform=None, num_processes=1):
    print(f"Loading folder: {folder_path}")
    files = list_files_by_type(folder_path, "jpg")
    if category is not None:
        files = [file for file in files if extract_index_and_category(file)[1] == category]

    # Use process_map from tqdm.contrib.concurrent for better tqdm updates
    results = thread_map(process_image, [(file, label, transform) for file in files], max_workers=num_processes, chunksize=1)

    x = torch.stack([result[0] for result in results])
    y = torch.stack([result[1] for result in results])
    return x, y

def loadImages():


    
    x1, y1 = load_folder("CIFAKE/train/REAL", 1, None, None, 4)
    x2, y2 = load_folder("CIFAKE/test/REAL", 1, None, None, 4)
    x1 = x1.to(fetchDevice())
    x2 = x2.to(fetchDevice())
    y1 = y1.to(fetchDevice())
    y2 = y2.to(fetchDevice())


    x3, y3 = load_folder("CIFAKE/train/FAKE", 0, None, None, 4)
    x4, y4 = load_folder("CIFAKE/test/FAKE", 0, None, None, 4)
    x3 = x3.to(fetchDevice())
    x4 = x4.to(fetchDevice())
    y3 = y3.to(fetchDevice())
    y4 = y4.to(fetchDevice())

    imageTensors = {
        "realImage": {
            "train": {
                "1": x1,
            },
            "test": {
                "1": x2
            }
        },
        "fakeImage": {
            "train": {
                "originalSD14": x3
            },
            "test": {
                "originalSD14": x4
            }
        }
    }
    labelTensors = {
        "realImage": {
            "train": {
                "1": y1,
            },
            "test": {
                "1": y2
            }
        },
        "fakeImage": {
            "train": {
                "originalSD14": y3
            },
            "test": {
                "originalSD14": y4
            }
        }
    }

    label_0_folders = list(os.listdir("STABLEDIFFUSION"))
    print(len(label_0_folders))
    print("sd21cifake-32pxjpg-n" in label_0_folders)
    for f in label_0_folders:
        if f[0] == '.' or f == "originalSD14":
            pass
        else:

            subFolders = list(os.listdir(f"STABLEDIFFUSION/{f}"))
            for sf in subFolders:
                if sf[0] == '.':
                    subFolders.remove(sf)
            
            trainX = [0]*len(subFolders)
            trainY = [0]*len(subFolders)
            testX = [0]*len(subFolders)
            testY = [0]*len(subFolders)
            index = 0
            for sf in subFolders:
                x1, y1 = load_folder(f"STABLEDIFFUSION/{f}/{sf}", 0, None, None, 4)
                indices = torch.randperm(x1.size()[0])
                x1=x1[indices]
                y1=y1[indices]

                trainX[index], testX[index] = torch.split(x1, [5000, 1000])
                trainY[index], testY[index] = torch.split(y1, [5000, 1000])
                index += 1

            imageTensors["fakeImage"]["train"][f] = torch.cat(tuple(trainX)).to(fetchDevice())
            imageTensors["fakeImage"]["test"][f] = torch.cat(tuple(testX)).to(fetchDevice())
            
            labelTensors["fakeImage"]["train"][f] = torch.cat(tuple(trainY)).to(fetchDevice())
            labelTensors["fakeImage"]["test"][f] = torch.cat(tuple(testY)).to(fetchDevice())

           


    return (imageTensors, labelTensors)


imageTensors, labelTensors = loadImages()




/Users/justinjiang/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading folder: CIFAKE/train/REAL


100%|██████████| 50000/50000 [00:06<00:00, 7619.26it/s]


Loading folder: CIFAKE/test/REAL


100%|██████████| 10000/10000 [00:01<00:00, 7763.81it/s]


...

100%|██████████| 6000/6000 [00:00<00:00, 6773.06it/s]


Loading folder: STABLEDIFFUSION/sd30cifake-32jpg/SD30Truck


100%|██████████| 6000/6000 [00:00<00:00, 7870.80it/s]


Loading folder: STABLEDIFFUSION/sd30cifake-32jpg/SD30Frog


100%|██████████| 6000/6000 [00:00<00:00, 7776.81it/s]


In [2]:
for i in imageTensors['fakeImage']['train']:
    print(i)

print("sd21cifake-32pxjpg-n" in imageTensors['fakeImage']['train'])

originalSD14
sd21-rpoa-n-32jpg
sd21cifake-32pxjpg-n
sd21-rpoa-32jpg
lora-sd21-v3-32JPG
lora-sd21-v1-32jpg
sd21-gpt40p-32jpg
sd21-apoa-32jpg
d21cifake-512pxjpg
sd21cifake-32jpg
sd21-apoan-32jpg
sd21cifake-512pxjpg-n
sd30-dmp32jpg
sd30cifake-32jpg
True


In [3]:
from torch.utils.data import Dataset
print(imageTensors)

class CIFAKEDataset2(Dataset):
        
    def __init__(self, fakeSetName, train): 
        global imageTensors
        global labelTensors
        train = "train" if train else "test"
        fakeImage = imageTensors["fakeImage"][train][fakeSetName]
        fakeLabels = labelTensors["fakeImage"][train][fakeSetName]
        realImage = imageTensors["realImage"][train]["1"]
        realLabels = labelTensors["realImage"][train]["1"]

        self.x = torch.cat((fakeImage, realImage))
        self.y = torch.cat((fakeLabels, realLabels))

        indices = torch.randperm(self.x.size()[0])
        self.x = self.x[indices].to(fetchDevice())
        self.y = self.y[indices].to(fetchDevice())
        

    def __len__(self):
        return self.y.size(0)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
    def data_dim(self):
        return self.x[0].size()

{'realImage': {'train': {'1': tensor([[[[0.2353, 0.2275, 0.2235,  ..., 0.2000, 0.1843, 0.1725],
          [0.2627, 0.2431, 0.2157,  ..., 0.1961, 0.2196, 0.2235],
          [0.1686, 0.1961, 0.2392,  ..., 0.2549, 0.2078, 0.1529],
          ...,
          [0.4824, 0.4980, 0.4980,  ..., 0.2824, 0.2549, 0.2353],
          [0.4039, 0.4078, 0.4039,  ..., 0.2941, 0.2745, 0.2510],
          [0.4196, 0.4157, 0.4000,  ..., 0.4196, 0.3961, 0.3843]],

         [[0.2392, 0.2314, 0.2157,  ..., 0.2549, 0.2431, 0.2314],
          [0.2863, 0.2588, 0.2275,  ..., 0.2431, 0.2745, 0.2784],
          [0.2157, 0.2392, 0.2745,  ..., 0.2941, 0.2471, 0.2000],
          ...,
          [0.4745, 0.4824, 0.4824,  ..., 0.2941, 0.2667, 0.2549],
          [0.3961, 0.4000, 0.3882,  ..., 0.3373, 0.3176, 0.3020],
          [0.4118, 0.4078, 0.3882,  ..., 0.4824, 0.4667, 0.4549]],

         [[0.1686, 0.1686, 0.1686,  ..., 0.1255, 0.1137, 0.1020],
          [0.1843, 0.1686, 0.1529,  ..., 0.1490, 0.1765, 0.1804],
          [0

In [4]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import os
import fnmatch
import re


import numpy as np
from CIFAKEClassifier import CIFAKEClassifier
from device import fetchDevice
import dataloader 
from dataloader import CIFAKEDataset


class DatasetClassifier:
    def __init__(self, fakeFolder):
        self.fakeFolder = fakeFolder
        self.model = CIFAKEClassifier()
        self.model = self.model.to(fetchDevice())

        # Hyperparameters
        self.batch_size = 1000
        self.learning_rate = 1e-3
        self.epochs = 15

    def train(self):
        # Dataloader for batch training
        trainSet = CIFAKEDataset2(self.fakeFolder, True)
        train_loader = DataLoader(dataset=trainSet, batch_size=self.batch_size, shuffle=True)
        #self.test_loader = DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=False)

        # Loss function and optimizer
        criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
        optimizer = Adam(self.model.parameters(), lr=self.learning_rate)
        self.model.train()
        for epoch in range(self.epochs):
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data
                labels = labels.float()  # BCELoss expects labels to be in float format

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = self.model(inputs).squeeze()  # Remove unnecessary dimensions
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 10 == 9:  # Print every 10 mini-batches
                    print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 10:.4f}')
                    running_loss = 0.0

        torch.save(self.model.state_dict(), 'weights/' + self.fakeFolder + '_model.pth')
        print('Finished Training ' + self.fakeFolder)

    def loadWeight(self, weightPath):
        self.model.load_state_dict(torch.load(weightPath))

    def fetchTestDataSet(self):
        testSet = CIFAKEDataset2(self.fakeFolder, False)
        return DataLoader(dataset=testSet, batch_size=self.batch_size, shuffle=False)
    
    
    def test_model(self, datasetClassifier):
        self.model.eval()
        TP = 0
        FP = 0
        TN = 0
        FN = 0

        with torch.no_grad():
            for data in datasetClassifier.fetchTestDataSet():
                images, labels = data
                outputs = self.model(images).squeeze()  # Remove unnecessary dimensions
                predicted = torch.round(outputs).tolist()  # Round to get binary predictions
                labels = labels.tolist()
                for i in range(len(predicted)):
                    if labels[i] == 1 and predicted[i] == 1:
                        TP += 1
                    elif labels[i] == 1 and predicted[i] == 0:
                        FN += 1
                    elif labels[i] == 0 and predicted[i] == 1:
                        FP += 1
                    elif labels[i] == 0 and predicted[i] == 0:
                        TN += 1
                    else:
                        print("\n\n\n \t\t\tERRROR \n\n\n")



        return (TP, FP, TN, FN)
        
    def getName(self):
        return self.fakeFolder

In [5]:
#comment

In [6]:
fakeDatasets = list(os.listdir("STABLEDIFFUSION"))
datasetClassifiers = [0]*len(fakeDatasets)
counter = 0
fakeDatasets.remove(".DS_Store")
for i in fakeDatasets:
    
    datasetClassifiers[counter] = DatasetClassifier(f"{i}")
    datasetClassifiers[counter].train()
    counter += 1
    print(f"""\n
        !!!!!!!!!!!!!!!!!!!!!!
        CURRENTLY AT {counter}
        !!!!!!!!!!!!!!!!!!!!!!
        \n""")
    

Epoch 1, Batch 10, Loss: 0.6804
Epoch 1, Batch 20, Loss: 0.6098
Epoch 1, Batch 30, Loss: 0.5141
Epoch 1, Batch 40, Loss: 0.4455
Epoch 1, Batch 50, Loss: 0.4162
Epoch 1, Batch 60, Loss: 0.3875
Epoch 1, Batch 70, Loss: 0.3621
Epoch 1, Batch 80, Loss: 0.3546
Epoch 1, Batch 90, Loss: 0.3330
Epoch 1, Batch 100, Loss: 0.3131
Epoch 2, Batch 10, Loss: 0.2995
Epoch 2, Batch 20, Loss: 0.2876
Epoch 2, Batch 30, Loss: 0.2787
Epoch 2, Batch 40, Loss: 0.2621
Epoch 2, Batch 50, Loss: 0.2567
Epoch 2, Batch 60, Loss: 0.2441
Epoch 2, Batch 70, Loss: 0.2361
Epoch 2, Batch 80, Loss: 0.2317
Epoch 2, Batch 90, Loss: 0.2171
Epoch 2, Batch 100, Loss: 0.2236
Epoch 3, Batch 10, Loss: 0.2132
Epoch 3, Batch 20, Loss: 0.2153
Epoch 3, Batch 30, Loss: 0.2066
Epoch 3, Batch 40, Loss: 0.2361
Epoch 3, Batch 50, Loss: 0.2047
Epoch 3, Batch 60, Loss: 0.1922
Epoch 3, Batch 70, Loss: 0.1845
Epoch 3, Batch 80, Loss: 0.2006
Epoch 3, Batch 90, Loss: 0.1999
Epoch 3, Batch 100, Loss: 0.1937
Epoch 4, Batch 10, Loss: 0.1771
Epoch

In [7]:
datasetClassifiers.remove(0)

In [8]:
def formatCell(text: str) -> str:
    initLen = len(text)
    if(initLen < 72):
        text += " "*(72 - initLen)
    else:
        text = text[:72]
    text += "|"
    return text

def formatNum(num) -> str:
    formatted = str(num)[:6]
    if(len(formatted) < 6):
        formatted += " " * (6 - len(formatted))
    return formatted


tableLabelTop = formatCell("")
for trainedModel in range (len(datasetClassifiers)):
    tableLabelTop += formatCell(datasetClassifiers[trainedModel].getName())
print(tableLabelTop)
print(len(tableLabelTop)*"_")
for trainedModel in range (len(datasetClassifiers)):
    resultPrint: str = formatCell(datasetClassifiers[trainedModel].getName())
    for testDataSet in range(len(datasetClassifiers)):
        TP, FP, TN, FN = datasetClassifiers[trainedModel].test_model(datasetClassifiers[testDataSet])
        accuracy:float = float(TP + TN)/float(TP + FP + TN + FN)
        prescission:float = float(TP)/float(TP + FP)
        recall:float = float(TP)/float(TP + FN) 
        falseAccuracy:float = float(TN)/float(TN + FP)
        trueAccuracy:float = float(TP)/float(TP + FN)
        resultPrint += formatCell(f"({formatNum(TP)}, {formatNum(FP)}, {formatNum(TN)}, {formatNum(FN)}, {formatNum(accuracy)}, {formatNum(prescission)}, {formatNum(recall)}, {formatNum(falseAccuracy)}, {formatNum(trueAccuracy)})")
    print(resultPrint)
    print(len(resultPrint)*"_")


                                                                        |sd21-rpoa-n-32jpg                                                       |sd21cifake-32pxjpg-n                                                    |sd21-rpoa-32jpg                                                         |lora-sd21-v3-32JPG                                                      |lora-sd21-v1-32jpg                                                      |sd21-gpt40p-32jpg                                                       |sd21-apoa-32jpg                                                         |d21cifake-512pxjpg                                                      |sd21cifake-32jpg                                                        |originalSD14                                                            |sd21-apoan-32jpg                                                        |sd21cifake-512pxjpg-n                                                   |sd30-dmp32jpg                                      

In [9]:
print(len("sd21cifake-512pxjpg-n"))

21
